In [1]:
import numpy as np
from datetime import datetime
import os
import itertools as it
from env import *
import pandas as pd
import random
from Beurele_agent import *
from MO_agent import *


In [2]:
e=tiger_POMDP_env(read_config=True,config_address='./tiger.json',parameters=None)
num_to_act=dict(zip(list(e.actions.values()),list(e.actions.keys())))
e.discount_factor=0.9

{'state_change_r2l': 0.1, 'state_change_l2r': 0.1, 'false_observation_get_l_while_r': 0.3, 'false_observation_get_r_while_l': 0.3, 'reward_listen': -0.5, 'reward_low_incorrect': -1, 'reward_high_incorrect': -2, 'reward_low_correct': 1, 'reward_high_correct': 2, 'discount_factor': 0.9, 'initial_wealth': 0}


In [3]:
def check_inc_dec(weights,exp_vorfaktoren):
    points=np.linspace(-10,10,100)
    y=np.zeros(len(points))
    for w,weight in enumerate(weights):
        y=y+(weight*np.exp(exp_vorfaktoren[w]*points))
    direction=0
    top=len(y)-1
    for i in range(len(y)-1):
        if y[i]<=y[i+1]:
            direction+=1
        elif y[i]>=y[i+1]:
            direction-=1
        else:
            top=top-1
    if direction==top:
        return 1
    elif direction==-top:
        return -1
    else:
        return 0

### Check similarity of methods' actions

In [6]:
E=[-1.5,-0.2,-0.1,0.1,0.2,1.5]
W=[-1,0,1,2]
inits=[[1,0],[0.9,0.1],[0.75,0.25],[0.5,0.5]]
initial_observation=0

loop_counts=0
for max_depth in range(1,3):
    maxdiff=[]
    incdec=[]
    corr_count=0
    all_count=0
    loop_counts+=1
    print(loop_counts)
    for e1 in E:
        for e2 in E:
            exp_vorfaktoren=[e1,e2]
            for w1 in W:
                for w2 in W:
                    exp_weights=[w1,w2]
                    #incdec.append(check_inc_dec(exp_weights,exp_vorfaktoren))
                    #if check_inc_dec(exp_weights,exp_vorfaktoren)==0:
                    #    break
                    for init in inits:
                        ag2=Multi_Variate_agent(environment=e, planning_depth=max_depth, partitioning_chunk_number=10,agent_mode='cheating')
                        ag2.reset()
                        x_map2,M2,F2,G2,X2,value_function2,all_theta2=ag2.pre_planning(exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights, initial_theta=init,initial_observation=initial_observation)
                        v2,a2,q2,vf2=ag2.value_iteration()
                        
                        
                        ag=Bauerle_Rieder_agent(environment=e, num_of_Mu_chunks=2,max_iterations=max_depth)
                        b_S,b_r,b_map,b_q_func,b_value_function,b_action_func=ag.continious_optimized_planning(initial_mu_state=init,initial_observation=initial_observation,initial_wealth=0,exp_weights=exp_weights,exp_vorfaktoren=exp_vorfaktoren)
                        
                        #m_r,m_map, m_q_func,m_value_function,m_action_func=MO_cheating(env_dynamics=e,planning_depth=max_depth,exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights ,initial_theta=init,initial_observation=0,rounding_prec_coeff=10000)
                        #b_S,b_r,b_map,b_q_func,b_value_function,b_action_func=Beurele_cheating(env_dynamics=e,planning_depth=max_depth,exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights ,initial_mu_state=init,initial_wealth=0,comparison_precision=1.0e-5)
                        
                        mx_init=init*len(init)
                        mx_init.append(0)
                        mr_init=[0]*len(init)
                        m_init=(tuple(mx_init),tuple(mr_init),0)
                        b_init=(initial_observation,tuple(init),0)
                        all_count+=1
                        if a2[0][tuple(m_init)]==b_action_func[0][b_init]:
                            corr_count+=1
                        else:
                            
                            pass
                        aa=np.abs(np.subtract(q2[0][tuple(m_init)],b_q_func[0][tuple(b_init)]))
                        bb=np.add(np.abs(q2[0][tuple(m_init)]),np.abs(b_q_func[0][tuple(b_init)]))+0.1
                        mdiff=2*np.divide(aa,bb)                       
                        maxdiff.extend(mdiff)
                         
                        #if mdiff>0.01:
                        #    maxdiff.append(mdiff)
                            
                            #print (q2[0][tuple(m_init)], b_q_func[0][tuple(b_init)],'           ',a2[0][tuple(m_init)],b_action_func[0][0][tuple(init)],'    ||     exp:',exp_vorfaktoren,exp_weights,init)
    print('--------------------------')                        
    print(all_count-corr_count,all_count,str(corr_count*100/all_count)+'%')
    
    maxdiff=(np.array(maxdiff)*1000).astype(np.int64)/1000.0
    total_cases=len(maxdiff)
    threshs=[0.01,0.05,0.1,0.2,0.5,1]
    false_rate=np.zeros(len(threshs))
    for th,threshold in enumerate(threshs) :

        false_rate[th]=len(np.where(maxdiff>threshold)[0])/total_cases
    #print(threshs)                       
    print(false_rate*100)
    
    print('=====================================')

1
--------------------------
0 1280 100.0%
[0. 0. 0. 0. 0. 0.]
2
--------------------------
1 1280 99.921875%
[20.796875 16.375     0.421875  0.203125  0.046875  0.046875]


## Simulation

### Beurele

In [3]:
max_depth=2
planning_depth=max_depth
exp_vorfaktoren=[-1,0.1]
exp_weights=[-1,1]
initial_Mu=[0.5,0.5]
initial_observation=0
initial_wealth=0

In [4]:
###############    Environment
e.reset()
initial_observation=e.current_state
observation=initial_observation
state=e.current_state
##############       Agent
# Create the agent
ag=Bauerle_Rieder_agent(environment=e, num_of_Mu_chunks=2,max_iterations=max_depth)

# Run its value iteration
b_S,b_reachables,b_map,b_q_func,b_value_function,b_action_func=ag.continious_optimized_planning(initial_mu_state=initial_Mu,initial_observation=initial_observation,initial_wealth=0,exp_weights=exp_weights,exp_vorfaktoren=exp_vorfaktoren)

# reset the agent 
ag.ch_reset(initial_mu_state=initial_Mu,initial_observation=initial_observation,initial_wealth=initial_wealth)

#############         Simulation

for t in range(planning_depth):
    
    print('t=',t)
    print('------')
    print('state:',state,e.states[state])
    print('last_observation:',observation)
    print('-------------------------------')
    print('')
    print('current internal state:',ag.current_internal_state)
    print('')
    
    # agent select the action
    action,value_of_action,_=ag.ch_do_action()
    
    #environment feedback
    t1,t2,state,reward,observation=e.step(num_to_act[action])
    
    # agent update
    new_x=ag.ch_update_agent(new_observation=observation) 
    
    print('             action:',num_to_act[action])  
    print ('             **' )
    print('reward:',reward,'new observation:',observation)
    print('')
    print('new internal (Mu) state:', ag.current_internal_state)
    print('')
    print('============================')

t= 0
------
state: 1 tiger_left
last_observation: 1
-------------------------------

current internal state: (1, (0.5, 0.5), 0)

             action: listen
             **
reward: -0.5 new observation: 0

new internal (Mu) state: (0, (0.0, 0.0, 0.7, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0), 1)

t= 1
------
state: 1 tiger_left
last_observation: 0
-------------------------------

current internal state: (0, (0.0, 0.0, 0.7, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0), 1)

             action: open_left_low
             **
reward: -1.0 new observation: 0

new internal (Mu) state: (0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.35, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.35, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 2)



### Multi Variate (Multi Objective)

In [5]:
planning_depth=2
partitioning_chunk_number=1000
exp_vorfaktoren=[-1,0.1]
exp_weights=[-1,1]
initial_theta=[0.5,0.5]
initial_observation=0
modes=['discrete_optimized','naive','cheating']

In [6]:
###############    Environment

e.reset()
initial_observation=e.current_state
observation=initial_observation
state=e.current_state

##############     Agent 

ag=Multi_Variate_agent(environment=e, planning_depth=planning_depth, partitioning_chunk_number=partitioning_chunk_number,agent_mode='cheating')
ag.reset()
mo_x_map,mo_M,mo_F,mo_G,mo_reachables,mo_value_function_zero,mo_all_thetas=ag.pre_planning(exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights, initial_theta=initial_theta,initial_observation=initial_observation,initial_wealth=0)
mo_value_func,mo_action_func,mo_q_func,vf2=ag.value_iteration()

In [7]:




##############       Simulation
for t in range(planning_depth):
    
    print('t=',t)
    print('------')
    print('state:',state,e.states[state])
    print('last_observation:',observation)
    print('-------------------------------')
    print('')
    print('current internal state:',ag.current_internal_state)
    print('')
    
    # agent select the action
    action,value_of_action=ag.do_action()
    
    #environment feedback
    t1,t2,state,reward,observation=e.step(num_to_act[action])
    
    # agent update
    new_x=ag.update_agent(new_observation=observation) 
    
    print('             action:',num_to_act[action])  
    print ('             **' )
    print('reward:',reward,'new observation:',observation)
    print('')
    print('new internal state:', ag.current_internal_state)
    print('')
    print('============================')

t= 0
------
state: 1 tiger_left
last_observation: 1
-------------------------------

current internal state: ((0.5, 0.5, 0.5, 0.5, 1), (0, 0), 0)

             action: listen
             **
reward: -0.5 new observation: 1

new internal state: ((0.3, 0.7, 0.3, 0.7, 1), (-0.5, -0.5), 1)

t= 1
------
state: 1 tiger_left
last_observation: 1
-------------------------------

current internal state: ((0.3, 0.7, 0.3, 0.7, 1), (-0.5, -0.5), 1)

             action: open_right_low
             **
reward: 1.0 new observation: 0

new internal state: ((0.5, 0.5, 0.5, 0.5, 0), (-0.5634127493, -0.103237705), 2)

